# Creating a STAC using PySTAC

Notebook going through the basic of creating a SpatioTemporal Asset Catalog using Python 3. Full details at are in the documentation for PySTAC 

## Install and load 

In [6]:
pip install pystac[validation] # install pystac

You should consider upgrading via the '/Users/oskarfkrauss/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pystac

## Obtain data and metadata 

In [1]:
import csv
import gzip
from io import StringIO
from urllib.request import urlopen

# Collection 1 data
url = 'https://landsat-pds.s3.amazonaws.com/c1/L8/scene_list.gz'

# Read and unzip the content
response = urlopen(url)
gunzip_response = gzip.GzipFile(fileobj=response)
content = gunzip_response.read()

# Read the scenes in as dictionaries
scenes = list(csv.DictReader(StringIO(content.decode("utf-8"))))

In [2]:
len(scenes) # lots of them!

2356735

In [3]:
scenes[0] # look at what scenes are

{'productId': 'LC08_L1TP_149039_20170411_20170415_01_T1',
 'entityId': 'LC81490392017101LGN00',
 'acquisitionDate': '2017-04-11 05:36:29.349932',
 'cloudCover': '0.0',
 'processingLevel': 'L1TP',
 'path': '149',
 'row': '39',
 'min_lat': '29.22165',
 'min_lon': '72.41205',
 'max_lat': '31.34742',
 'max_lon': '74.84666',
 'download_url': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/149/039/LC08_L1TP_149039_20170411_20170415_01_T1/index.html'}

Filter to ones which contain a certain location

In [4]:
lat, lon = 55.9422, -3.1823
location_name = "Edinburgh Flat"
filter_year = '2020'

In [5]:
def keep_scene(scene): # function that does the filtering
    contains_location = float(scene['min_lat']) < lat and float(scene['max_lat']) > lat and \
                        float(scene['min_lon']) < lon and float(scene['max_lon']) > lon
    is_correct_year = '{}-'.format(filter_year) in scene['acquisitionDate']
    return contains_location and is_correct_year
        
location_scenes = [scene for scene in scenes if keep_scene(scene)]

In [6]:
len(location_scenes) # fewer of them, woop

58

In [7]:
scene = location_scenes[0]
scene

{'productId': 'LC08_L1GT_205021_20200106_20200106_01_RT',
 'entityId': 'LC82050212020006LGN00',
 'acquisitionDate': '2020-01-06 11:16:07.471447',
 'cloudCover': '94.11',
 'processingLevel': 'L1GT',
 'path': '205',
 'row': '21',
 'min_lat': '54.78572',
 'min_lon': '-5.63922',
 'max_lat': '56.97768',
 'max_lon': '-1.73361',
 'download_url': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/205/021/LC08_L1GT_205021_20200106_20200106_01_RT/index.html'}

Now extract metadata (additional data about scenes) from MTL file. This file is alongside the `download_url` and so we need a funciton to extract necessary pieces (from scenes) and then retrive MTL. 

In [8]:
def get_asset_url(scene, suffix):
    product_id = scene['productId']
    download_url = scene['download_url']    
    asset_filename = '{}_{}'.format(product_id, suffix)
    return download_url.replace('index.html', asset_filename)

Now read url

In [11]:
mtl_url = get_asset_url(scene, 'MTL.txt')
print(pystac.STAC_IO.read_text(mtl_url))

GROUP = L1_METADATA_FILE
  GROUP = METADATA_FILE_INFO
    ORIGIN = "Image courtesy of the U.S. Geological Survey"
    REQUEST_ID = "0502001063447_00003"
    LANDSAT_SCENE_ID = "LC82050212020006LGN00"
    LANDSAT_PRODUCT_ID = "LC08_L1GT_205021_20200106_20200106_01_RT"
    COLLECTION_NUMBER = 01
    FILE_DATE = 2020-01-06T15:25:28Z
    STATION_ID = "LGN"
    PROCESSING_SOFTWARE_VERSION = "LPGS_13.1.0"
  END_GROUP = METADATA_FILE_INFO
  GROUP = PRODUCT_METADATA
    DATA_TYPE = "L1GT"
    COLLECTION_CATEGORY = "RT"
    ELEVATION_SOURCE = "GLS2000"
    OUTPUT_FORMAT = "GEOTIFF"
    SPACECRAFT_ID = "LANDSAT_8"
    SENSOR_ID = "OLI_TIRS"
    WRS_PATH = 205
    WRS_ROW = 21
    NADIR_OFFNADIR = "NADIR"
    TARGET_WRS_PATH = 205
    TARGET_WRS_ROW = 21
    DATE_ACQUIRED = 2020-01-06
    SCENE_CENTER_TIME = "11:16:07.4714479Z"
    CORNER_UL_LAT_PRODUCT = 56.97768
    CORNER_UL_LON_PRODUCT = -5.63922
    CORNER_UR_LAT_PRODUCT = 56.99834
    CORNER_UR_LON_PRODUCT = -1.66005
    CORNER_LL_LAT_PRODU

In [12]:
def get_metadata(url):
    """ Convert Landsat MTL file to dictionary of metadata values """
    mtl = {}
    mtl_text = pystac.STAC_IO.read_text(url)
    for line in mtl_text.split('\n'):
        meta = line.replace('\"', "").strip().split('=')
        if len(meta) > 1:
            key = meta[0].strip()
            item = meta[1].strip()
            if key != "GROUP" and key != "END_GROUP":
                mtl[key] = item
    return mtl


In [13]:
metadata = get_metadata(mtl_url)
metadata

{'ORIGIN': 'Image courtesy of the U.S. Geological Survey',
 'REQUEST_ID': '0502001063447_00003',
 'LANDSAT_SCENE_ID': 'LC82050212020006LGN00',
 'LANDSAT_PRODUCT_ID': 'LC08_L1GT_205021_20200106_20200106_01_RT',
 'COLLECTION_NUMBER': '01',
 'FILE_DATE': '2020-01-06T15:25:28Z',
 'STATION_ID': 'LGN',
 'PROCESSING_SOFTWARE_VERSION': 'LPGS_13.1.0',
 'DATA_TYPE': 'L1GT',
 'COLLECTION_CATEGORY': 'RT',
 'ELEVATION_SOURCE': 'GLS2000',
 'OUTPUT_FORMAT': 'GEOTIFF',
 'SPACECRAFT_ID': 'LANDSAT_8',
 'SENSOR_ID': 'OLI_TIRS',
 'WRS_PATH': '205',
 'WRS_ROW': '21',
 'NADIR_OFFNADIR': 'NADIR',
 'TARGET_WRS_PATH': '205',
 'TARGET_WRS_ROW': '21',
 'DATE_ACQUIRED': '2020-01-06',
 'SCENE_CENTER_TIME': '11:16:07.4714479Z',
 'CORNER_UL_LAT_PRODUCT': '56.97768',
 'CORNER_UL_LON_PRODUCT': '-5.63922',
 'CORNER_UR_LAT_PRODUCT': '56.99834',
 'CORNER_UR_LON_PRODUCT': '-1.66005',
 'CORNER_LL_LAT_PRODUCT': '54.78572',
 'CORNER_LL_LON_PRODUCT': '-5.49447',
 'CORNER_LR_LAT_PRODUCT': '54.80475',
 'CORNER_LR_LON_PRODUCT': 

Now have everything me need to make STAC item...

## Make STAC Item 

Items have several required fields including `id`, `datetime`, and `bbox` (bounding box) so we first need to extract those from the metadata. This can be done using the following functions.  

In [14]:
# id

def get_id(metadata): 
    return metadata['LANDSAT_SCENE_ID'][:-5]

# datetime

from dateutil.parser import parse

def get_datetime(metadata):
    return parse('%sT%s' % (metadata['DATE_ACQUIRED'], metadata['SCENE_CENTER_TIME']))

# bbox

def get_bbox(metadata):
    coords = [[
        [float(metadata['CORNER_UL_LON_PRODUCT']), float(metadata['CORNER_UL_LAT_PRODUCT'])],
        [float(metadata['CORNER_UR_LON_PRODUCT']), float(metadata['CORNER_UR_LAT_PRODUCT'])],
        [float(metadata['CORNER_LR_LON_PRODUCT']), float(metadata['CORNER_LR_LAT_PRODUCT'])],
        [float(metadata['CORNER_LL_LON_PRODUCT']), float(metadata['CORNER_LL_LAT_PRODUCT'])],
        [float(metadata['CORNER_UL_LON_PRODUCT']), float(metadata['CORNER_UL_LAT_PRODUCT'])]
    ]]
    lats = [c[1] for c in coords[0]]
    lons = [c[0] for c in coords[0]]
    return [min(lons), min(lats), max(lons), max(lats)]

# geometry, this is a bit trickier since landsat tiles are tilted

def get_geometry(scene, bbox):
    url = get_asset_url(scene, 'ANG.txt')
    sz = []
    coords = []
    ang_text = pystac.STAC_IO.read_text(url)
    for line in ang_text.split('\n'):
        if 'BAND01_NUM_L1T_LINES' in line or 'BAND01_NUM_L1T_SAMPS' in line:
            sz.append(float(line.split('=')[1]))
        if 'BAND01_L1T_IMAGE_CORNER_LINES' in line or 'BAND01_L1T_IMAGE_CORNER_SAMPS' in line:
            coords.append([float(l) for l in line.split('=')[1].strip().strip('()').split(',')])
        if len(coords) == 2:
            break
    dlon = bbox[2] - bbox[0]
    dlat = bbox[3] - bbox[1]
    lons = [c/sz[1] * dlon + bbox[0] for c in coords[1]]
    lats = [((sz[0] - c)/sz[0]) * dlat + bbox[1] for c in coords[0]]
    coordinates = [[
        [lons[0], lats[0]], [lons[1], lats[1]], [lons[2], lats[2]], [lons[3], lats[3]], [lons[0], lats[0]]
    ]]
    
    return {'type': 'Polygon', 'coordinates': coordinates}

Then store variables...

In [15]:
item_id = get_id(metadata)
print('ID:', item_id)

item_datetime = get_datetime(metadata)
print('datetime:', item_datetime)

item_bbox = get_bbox(metadata)
print('bbox:', item_bbox)

item_geometry = get_geometry(scene, item_bbox) # from scene as opposed to metadata
print('geometry:', item_geometry)

ID: LC82050212020006
datetime: 2020-01-06 11:16:07.471447+00:00
bbox: [-5.63922, 54.78572, -1.66005, 56.99834]
geometry: {'type': 'Polygon', 'coordinates': [[[-4.679899357070097, 56.99739750867762], [-1.6618520817312392, 56.48293191696963], [-2.6246412167403794, 54.78628975250376], [-5.637228781570707, 55.30986728082811], [-4.679899357070097, 56.99739750867762]]]}


Can now double check we have the correct area by formatting the item geometry as GeoJSON then copying into a GeoJSON reader (such as http://geojson.io).

In [16]:
import json

print(json.dumps(item_geometry, indent=2))

{
  "type": "Polygon",
  "coordinates": [
    [
      [
        -4.679899357070097,
        56.99739750867762
      ],
      [
        -1.6618520817312392,
        56.48293191696963
      ],
      [
        -2.6246412167403794,
        54.78628975250376
      ],
      [
        -5.637228781570707,
        55.30986728082811
      ],
      [
        -4.679899357070097,
        56.99739750867762
      ]
    ]
  ]
}


No we have everything we need to make a STAC item! We create it below using pystac.

In [17]:
item = pystac.Item(id = item_id,
                  datetime = item_datetime,
                  bbox = item_bbox,
                  geometry = item_geometry,
                  properties = {})

In [18]:
item.validate() #PySTAC function to check we're doing things correctly

No errors, all good!

So far we only have the required attributes, however it is often a good idea to add other common features, such as ground sample distance. We also enable the `eo` extenstion (not 100% on this) so that we can include the cloud cover data.

In [19]:
item.common_metadata.gsd = 30.0 # not all bands are 30.0 but we can adjust this when we add the asset later

In [20]:
item.ext.enable('eo')

def get_cloud_cover(metadata):
    return float(metadata['CLOUD_COVER'])

item.ext.eo.cloud_cover = get_cloud_cover(metadata)
item.ext.eo.cloud_cover

94.11

Now that we have set up the item, we can add assets, i.e. the actual files.

### Adding assets

First provide information on Landsat band info

In [21]:
from pystac.extensions.eo import Band

landsat_band_info = {
    'B1': {
        'band': Band.create(name="B1", common_name="coastal", center_wavelength=0.48, full_width_half_max=0.02),
        'gsd': 30.0
    },
    'B2': {
        'band': Band.create(name="B2", common_name="blue", center_wavelength=0.44, full_width_half_max=0.06),
        'gsd': 30.0
    },
    'B3': {
        'band': Band.create(name="B3", common_name="green", center_wavelength=0.56, full_width_half_max=0.06),
        'gsd': 30.0
    },
    'B4': {
        'band': Band.create(name="B4", common_name="red", center_wavelength=0.65, full_width_half_max=0.04),
        'gsd': 30.0
    },
    'B5': {
        'band': Band.create(name="B5", common_name="nir", center_wavelength=0.86, full_width_half_max=0.03),
        'gsd': 30.0
    },
    'B6': {
        'band': Band.create(name="B6", common_name="swir16", center_wavelength=1.6, full_width_half_max=0.08),
        'gsd': 30.0
    },
    'B7': {
        'band': Band.create(name="B7", common_name="swir22", center_wavelength=2.2, full_width_half_max=0.2),
        'gsd': 30.0
    },
    'B8': {
        'band': Band.create(name="B8", common_name="pan", center_wavelength=0.59, full_width_half_max=0.18),
        'gsd': 15.0
    },
    'B9': {
        'band': Band.create(name="B9", common_name="cirrus", center_wavelength=1.37, full_width_half_max=0.02),
        'gsd': 30.0
    },
    'B10': {
        'band': Band.create(name="B10", common_name="lwir11", center_wavelength=10.9, full_width_half_max=0.8),
        'gsd': 100.0
    },
    'B11': {
        'band': Band.create(name="B11", common_name="lwir12", center_wavelength=12.0, full_width_half_max=1.0),
        'gsd': 100.0
    }
}

Now define function to get other bandsat assets for each item

In [22]:
def get_other_assets(scene):
    return {
    'thumbnail': {
        'href': get_asset_url(scene, 'thumb_large.jpg'),
        'media_type': pystac.MediaType.JPEG
    },
    'index': {
        'href': get_asset_url(scene, 'index.html'),
        'media_type': 'application/html'
    },
    'ANG': {
        'href': get_asset_url(scene, 'ANG.txt'),
        'media_type': 'text/plain'
    },
    'MTL': {
        'href': get_asset_url(scene, 'MTL.txt'),
        'media_type': 'text/plain'
    },
    'BQA': {
        'href': get_asset_url(scene, 'BQA.TIF'),
        'media_type': pystac.MediaType.GEOTIFF
    }
}

We can now add relevant assets to items as follows

In [23]:
def add_assets(item, scene):
    # Add bands
    for band_id, band_info in landsat_band_info.items():
        band_url = get_asset_url(scene, '{}.TIF'.format(band_id))
        asset = pystac.Asset(href=band_url, media_type=pystac.MediaType.COG)
        bands = [band_info['band']]
        item.ext.eo.set_bands(bands, asset)
        item.add_asset(band_id, asset)
        
        # If this asset has a different GSD than the item, set it on the asset
        if band_info['gsd'] != item.common_metadata.gsd:
            item.common_metadata.set_gsd(band_info['gsd'], asset)
        
    # Add other assets
    for asset_id, asset_info in get_other_assets(scene).items():
        item.add_asset(asset_id, 
                       pystac.Asset(href=asset_info['href'], media_type=asset_info['media_type']))

In [24]:
add_assets(item, scene)

In [25]:
item.assets['B10'].to_dict()

{'href': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/205/021/LC08_L1GT_205021_20200106_20200106_01_RT/LC08_L1GT_205021_20200106_20200106_01_RT_B10.TIF',
 'type': <MediaType.COG: 'image/tiff; application=geotiff; profile=cloud-optimized'>,
 'eo:bands': [{'name': 'B10',
   'common_name': 'lwir11',
   'center_wavelength': 10.9,
   'full_width_half_max': 0.8}],
 'gsd': 100.0}

In [26]:
item.assets['thumbnail'].to_dict()

{'href': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/205/021/LC08_L1GT_205021_20200106_20200106_01_RT/LC08_L1GT_205021_20200106_20200106_01_RT_thumb_large.jpg',
 'type': <MediaType.JPEG: 'image/jpeg'>}

Can also add CRS as an extension to the item. To find CRS the below function determines EPSG code using centre lat of the scene.

In [27]:
def get_epsg(metadata, min_lat, max_lat):
    if 'UTM_ZONE' in metadata:
        center_lat = (min_lat + max_lat)/2.0
        return int(('326' if center_lat > 0 else '327') + metadata['UTM_ZONE'])
    else:
        return None

In [28]:
item.ext.enable('projection')
item.ext.projection.epsg = get_epsg(metadata, item.bbox[1], item.bbox[3]) 
item.ext.projection.epsg

32630

Another extenstion is view geometry which, can be founded and added as follows

In [29]:
def get_view_info(metadata):
    return { 'sun_azimuth': float(metadata['SUN_AZIMUTH']),
             'sun_elevation': float(metadata['SUN_ELEVATION']) }

In [30]:
item.ext.enable('view')
view_info = get_view_info(metadata)
item.ext.view.sun_azimuth = view_info['sun_azimuth']
item.ext.view.sun_elevation = view_info['sun_elevation']
item.properties

{'datetime': '2020-01-06T11:16:07.471447Z',
 'gsd': 30.0,
 'eo:cloud_cover': 94.11,
 'proj:epsg': 32630,
 'view:sun_azimuth': 164.99895314,
 'view:sun_elevation': 10.38957221}

Let's check that we have done everything correctly using `item.validate`

In [37]:
item.validate()

### Collections

Now that we know how to make an item, lets gather the items into a collection. 

In [39]:
collection_id = '{}-landsat-8'.format(location_name) # id for collection
collection_id

'Edinburgh Flat-landsat-8'

In [40]:
collection_title = '2020 Landsat 8 images over {}'.format(location_name) # title
collection_title

'2020 Landsat 8 images over Edinburgh Flat'

In [41]:
collection_description = '''### {} Landsat 8

A collection of Landsat 8 scenes around {} in 2020.
'''.format(location_name, location_name)
print(collection_description) # description for collection written in markdown

### Edinburgh Flat Landsat 8

A collection of Landsat 8 scenes around Edinburgh Flat in 2020.



Along with id, title, and description,we also need the temporal and spatial extent of the collection, i.e. the time range we have and the whole area the collection encompasses.

In [44]:
from datetime import datetime

spatial_extent = pystac.SpatialExtent([[-180, -90, 180, 90]])
temporal_extent = pystac.TemporalExtent([[datetime(2013, 6, 1), None]])
collection_extent = pystac.Extent(spatial_extent, temporal_extent)

In [45]:
collection = pystac.Collection(id=collection_id,
                     title=collection_title,
                     description=collection_description,
                     extent=collection_extent)

In [46]:
collection.to_dict()

{'id': 'Edinburgh Flat-landsat-8',
 'stac_version': '1.0.0-beta.2',
 'description': '### Edinburgh Flat Landsat 8\n\nA collection of Landsat 8 scenes around Edinburgh Flat in 2020.\n',
 'links': [{'rel': 'root', 'href': None, 'type': 'application/json'}],
 'title': '2020 Landsat 8 images over Edinburgh Flat',
 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
  'temporal': {'interval': [['2013-06-01T00:00:00Z', None]]}},
 'license': 'proprietary'}

In [49]:
collection.license = 'PDDL-1.0'

In [50]:
collection.to_dict()

{'id': 'Edinburgh Flat-landsat-8',
 'stac_version': '1.0.0-beta.2',
 'description': '### Edinburgh Flat Landsat 8\n\nA collection of Landsat 8 scenes around Edinburgh Flat in 2020.\n',
 'links': [{'rel': 'root', 'href': None, 'type': 'application/json'}],
 'title': '2020 Landsat 8 images over Edinburgh Flat',
 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
  'temporal': {'interval': [['2013-06-01T00:00:00Z', None]]}},
 'license': 'PDDL-1.0'}

### Looping through

Now that we have define the collection and created an item, we can loop through all scenes in the database and add them to the collection. The followinbg function condenses everything done above.

In [55]:
def item_from_scene(scene):
    
    mtl_url = get_asset_url(scene, 'MTL.txt')
    metadata = get_metadata(mtl_url)
    
    bbox = get_bbox(metadata)
    item = pystac.Item(id=get_id(metadata), 
                       datetime=get_datetime(metadata),
                       geometry=get_geometry(scene, bbox),
                       bbox=bbox,
                       properties={})
    
    item.common_metadata.gsd = 30.0
    
    item.ext.enable('eo')
    item.ext.eo.cloud_cover = get_cloud_cover(metadata)
    
    add_assets(item, scene)
    
    item.ext.enable('projection')
    item.ext.projection.epsg = get_epsg(metadata, item.bbox[1], item.bbox[3]) 

    item.ext.enable('view')
    view_info = get_view_info(metadata)
    item.ext.view.sun_azimuth = view_info['sun_azimuth']
    item.ext.view.sun_elevation = view_info['sun_elevation']

    item.validate()
    
    return item


In [56]:
for scene in location_scenes:
    item = item_from_scene(scene)
    collection.add_item(item)

With all these items we can update the extent of the collection, which we previously to be very broad.

In [57]:
collection.update_extent_from_items()
collection.extent.to_dict()

{'spatial': {'bbox': [[-5.70824, 54.78235, -0.07244, 56.99902]]},
 'temporal': {'interval': [['2020-01-06T11:16:07.471447Z',
    '2020-12-23T11:16:14.663021Z']]}}

So far we have been building everything in memory, now we need a href.

In [58]:
from pathlib import Path

root_path = str(Path.home() / '{}-landsat-stac'.format(location_name))

collection.normalize_hrefs(root_path)

<Collection id=Edinburgh Flat-landsat-8>

In [59]:
collection.validate_all()

### Write catalog

In [63]:
collection.save(pystac.CatalogType.SELF_CONTAINED)

In [61]:
collection.describe()

* <Collection id=Edinburgh Flat-landsat-8>
  * <Item id=LC82050212020006>
  * <Item id=LC82040212020015>
  * <Item id=LC82050212020022>
  * <Item id=LC82050212020022>
  * <Item id=LC82040212020031>
  * <Item id=LC82050212020038>
  * <Item id=LC82040212020031>
  * <Item id=LC82050212020038>
  * <Item id=LC82040212020047>
  * <Item id=LC82050212020054>
  * <Item id=LC82040212020047>
  * <Item id=LC82040212020063>
  * <Item id=LC82050212020070>
  * <Item id=LC82040212020063>
  * <Item id=LC82040212020079>
  * <Item id=LC82050212020086>
  * <Item id=LC82050212020070>
  * <Item id=LC82040212020079>
  * <Item id=LC82040212020095>
  * <Item id=LC82050212020086>
  * <Item id=LC82050212020102>
  * <Item id=LC82040212020111>
  * <Item id=LC82050212020102>
  * <Item id=LC82050212020118>
  * <Item id=LC82040212020127>
  * <Item id=LC82040212020111>
  * <Item id=LC82040212020127>
  * <Item id=LC82050212020134>
  * <Item id=LC82050212020118>
  * <Item id=LC82040212020143>
  * <Item id=LC820502120201

In [62]:
collection.to_dict()

{'id': 'Edinburgh Flat-landsat-8',
 'stac_version': '1.0.0-beta.2',
 'description': '### Edinburgh Flat Landsat 8\n\nA collection of Landsat 8 scenes around Edinburgh Flat in 2020.\n',
 'links': [{'rel': 'root',
   'href': './collection.json',
   'type': 'application/json'},
  {'rel': 'item',
   'href': './LC82050212020006/LC82050212020006.json',
   'type': 'application/json'},
  {'rel': 'item',
   'href': './LC82040212020015/LC82040212020015.json',
   'type': 'application/json'},
  {'rel': 'item',
   'href': './LC82050212020022/LC82050212020022.json',
   'type': 'application/json'},
  {'rel': 'item',
   'href': './LC82050212020022/LC82050212020022.json',
   'type': 'application/json'},
  {'rel': 'item',
   'href': './LC82040212020031/LC82040212020031.json',
   'type': 'application/json'},
  {'rel': 'item',
   'href': './LC82050212020038/LC82050212020038.json',
   'type': 'application/json'},
  {'rel': 'item',
   'href': './LC82040212020031/LC82040212020031.json',
   'type': 'applicati